In [11]:
import re
import bisect
from collections import defaultdict
from collections import Counter
import os
import pickle

import psycopg2
from unidecode import unidecode

import jetons

## Chargement des titres

In [2]:
connection = psycopg2.connect(dbname='loi', user='loi', password='baba')
curseur = connection.cursor()

curseur.execute("select id_, titre, titrefull from texte_version;")
resultats = curseur.fetchall()

curseur.close()
connection.close()

In [3]:
liste_id_ = [r[0] for r in resultats]
titres = [r[1] for r in resultats]
titres_entiers = [r[2] for r in resultats]

In [4]:
def trouve_id_(liste_index_titre, position):
    n_titre = bisect.bisect_right(liste_index_titre, position)-1
    return liste_id_[n_titre]

In [5]:
contenu_brut = ''.join([t + '\n' for t in titres_entiers])

In [6]:
liste_index_titre = [0] + [len(t) + 1 for t in titres_entiers]
for i in range(1, len(liste_index_titre)):
    liste_index_titre[i] += liste_index_titre[i-1]

In [7]:
position_mots, contenu_mots = jetons.transformation_jetons(contenu_brut)

## Comptage des n-grammes

In [25]:
nb_mots = 3

def mot_i(contenu_brut, position_mots_dans_paragraphe, contenu_mots_dans_paragraphe, i):
    d, f = position_mots_dans_paragraphe[i]
    mot = unidecode(contenu_brut[d:f]).lower()
    return mot

def construction_index_paragraphe(
    contenu_brut,
    position_mots_dans_paragraphe, contenu_mots_dans_paragraphe,
    index_passages
):
    id_ = trouve_id_(liste_index_titre, position_mots_dans_paragraphe[0][0])
        
    mots = [
        mot_i(contenu_brut, position_mots_dans_paragraphe, contenu_mots_dans_paragraphe, i)
        for i in range(min(nb_mots-1, len(position_mots_dans_paragraphe)))
    ]
    
    for i in range(nb_mots-1, len(position_mots_dans_paragraphe)):
        mots.append(mot_i(contenu_brut, position_mots_dans_paragraphe, contenu_mots_dans_paragraphe, i))

        if set(contenu_mots_dans_paragraphe[i-nb_mots+1:i+1]) == set([None]):
            passage = ' '.join(mots)
            index_passages[passage].add(id_)
        
        mots = mots[1:]

def construction_index_(contenu_brut, position_mots, contenu_mots, index_passages):
    i = 0
    for position_mots_dans_paragraphe, contenu_mots_dans_paragraphe in zip(position_mots, contenu_mots):
        construction_index_paragraphe(
            contenu_brut, position_mots_dans_paragraphe, contenu_mots_dans_paragraphe, index_passages)
        
        i += 1
        if i % 100000 == 0:
            print(i)
        

In [9]:
index_passages = defaultdict(set)
construction_index_(contenu_brut, position_mots, contenu_mots, index_passages)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000


In [12]:
dossier_data = '/home/michel/loi/data/'
with open(os.path.join(dossier_data, 'trigrammes.pickle'), 'wb') as f:
    pickle.dump(construction_index_, f)

In [26]:
n_grammes = set(list(index_passages.keys()))

In [27]:
len(n_grammes)

1244337

## Cherche les n-grammes dans le corpus

In [48]:
jeton_titre = '__titre'

def passe_paragraphe(
        contenu_brut,
        position_mots_dans_paragraphe, contenu_mots_dans_paragraphe
):
    est_n_gramme_dans_paragraphe = [False] * len(contenu_mots_dans_paragraphe)
    
    mots = [
        mot_i(contenu_brut, position_mots_dans_paragraphe, contenu_mots_dans_paragraphe, i)
        for i in range(min(nb_mots-1, len(position_mots_dans_paragraphe)))
    ]
    
    for i in range(nb_mots-1, len(position_mots_dans_paragraphe)):
        mots.append(mot_i(contenu_brut, position_mots_dans_paragraphe, contenu_mots_dans_paragraphe, i))

        if set(contenu_mots_dans_paragraphe[i-nb_mots+1:i+1]) == set([None]):
            n_gramme = ' '.join(mots)
            if n_gramme in n_grammes:
                est_n_gramme_dans_paragraphe[i-nb_mots+1:i+1] = [True] * nb_mots
        
        mots = mots[1:]

    return est_n_gramme_dans_paragraphe


In [49]:
def passe(contenu_brut, position_mots, contenu_mots):
    est_n_gramme = []
    for position_mots_dans_paragraphe, contenu_mots_dans_paragraphe in zip(position_mots, contenu_mots):
        est_n_gramme_dans_paragraphe = passe_paragraphe(
            contenu_brut,
            position_mots_dans_paragraphe, contenu_mots_dans_paragraphe)
        
        est_n_gramme.append(est_n_gramme_dans_paragraphe)
        
    return est_n_gramme

In [50]:
def joli_affichage(contenu_brut, position_mots, contenu_mots, est_n_gramme):
    for position_mots_dans_paragraphe, contenu_mots_dans_paragraphe, est_n_gramme_dans_paragraphe in zip(position_mots, contenu_mots, est_n_gramme):
        print(' | '.join(
            [
                contenu_brut[mot[0]:mot[1]] + (' (' + contenu + ')' if contenu else '') + (' (titre)' if eng else '')
                for mot, contenu, eng in zip(position_mots_dans_paragraphe, contenu_mots_dans_paragraphe, est_n_gramme_dans_paragraphe)
            ]))


In [53]:
"candidats ayant obtenu" in n_grammes

True

In [51]:
connection = psycopg2.connect(dbname='loi', user='loi', password='baba')
curseur = connection.cursor()

curseur.execute("select id_, bloc_textuel from article_2 limit 100;")
resultats = curseur.fetchall()

curseur.close()
connection.close()

liste_id_ = [r[0] for r in resultats]
liste_bloc_textuel = [r[1] for r in resultats]

liste_representations = ''
for id_, contenu_brut in resultats:
    position_mots, contenu_mots = jetons.transformation_jetons(contenu_brut)
    est_n_gramme = passe(contenu_brut, position_mots, contenu_mots)

    joli_affichage(contenu_brut, position_mots, contenu_mots, est_n_gramme)



L | article | L. 233-1 (__num_longue) | code | éducation | est | applicable (titre) | en (titre) | Polynésie (titre) | française (titre) | dans (titre) | sa (titre) | rédaction (titre) | résultant (titre) | article | 38 (__arabe) | loi | 22 juillet 2013 (__date) | susvisée | .
Il (titre) | est (titre) | créé (titre) | mention (titre) | complémentaire (titre) | « (titre) | technicien (titre) | équipements (titre) | audiovisuels (titre) | professionnels (titre) | » (titre) | dont | définition (titre) | et (titre) | conditions (titre) | délivrance (titre) | sont | fixées | conformément | dispositions | présent (titre) | arrêté (titre) | . (titre)
Ce (titre) | diplôme (titre) | est (titre) | classé | niveau | IV (__romain_maj) | nomenclature | interministérielle | niveaux | formation | .
L (titre) | autorisation (titre) | accordée (titre) | par (titre) | décision (titre) | n° (titre) | 2008-800 (__arabe_comp) | 8 juillet 2008 (__date) | susvisée | pour | exploitation | d (__lettre_min) | s

In [44]:
zip(position_mots_dans_paragraphe, contenu_mots_dans_paragraphe, est_trigramme)

NameError: name 'position_mots_dans_paragraphe' is not defined

In [ ]:

def passe(contenu_brut, position_mots, contenu_mots, nb_mots, est_numero, jeton):
    nouvelles_positions = []
    nouveaux_contenus = []
    for position_mots_dans_paragraphe, contenu_mots_dans_paragraphe in zip(position_mots, contenu_mots):
        nouvelles_positions_dans_paragraphe, nouveaux_contenus_dans_paragraphe = passe_paragraphe(
            contenu_brut,
            position_mots_dans_paragraphe, contenu_mots_dans_paragraphe,
            nb_mots, est_numero, jeton)
        
        nouvelles_positions.append(nouvelles_positions_dans_paragraphe)
        nouveaux_contenus.append(nouveaux_contenus_dans_paragraphe)
        
    return nouvelles_positions, nouveaux_contenus


In [94]:
tmp1 = [len(s) for c, s in index_passages.items()]

In [98]:
[c for c, s in index_passages.items() if len(s)>1000]

['etendue zones et',
 'et le gouvernement',
 'officiers publics ou',
 'pharmaceutiques agreees usage',
 'enseignement superieur et',
 'dispositions statutaires relatives',
 'portant promotion (',
 'but non lucratif',
 'relatif mise en',
 'administrateurs civils )',
 'du departement de',
 'civile professionnelle (',
 'nationale superieure techniques',
 'taux de l',
 'accidents du travail',
 'travaux publics etat',
 'l arrete du',
 'examen professionnel pour',
 'fixant la liste',
 'services transport aerien',
 'et agents maitrise',
 'cour comptes )',
 'dans le departement',
 'conclu dans cadre',
 'administration de la',
 'du ministere des',
 'ponts et chaussees',
 'portant reconduction autorisation',
 'agreees usage collectivites',
 ', de la',
 'loi de finances',
 'exploitation services transport',
 'agents non titulaires',
 'economie , finances',
 '1er de l',
 'de la famille',
 'et titularisation (',
 'pour acces corps',
 'zone speciale conservation',
 'nomination ( enseignement',
 'pol

In [95]:
Counter(tmp1)

Counter({1: 643378,
         2: 233261,
         3: 80516,
         4: 62399,
         5: 33023,
         6: 27994,
         7: 18471,
         8: 16139,
         9: 11644,
         10: 10314,
         11: 8025,
         12: 7380,
         13: 5998,
         14: 5510,
         15: 4658,
         16: 4311,
         17: 3639,
         18: 3483,
         19: 3058,
         20: 2861,
         21: 2538,
         22: 2311,
         23: 2082,
         24: 2026,
         25: 1791,
         26: 1835,
         27: 1653,
         28: 1504,
         29: 1347,
         30: 1369,
         31: 1159,
         32: 1102,
         33: 1049,
         34: 1045,
         35: 1009,
         36: 907,
         37: 860,
         38: 803,
         39: 770,
         40: 694,
         41: 739,
         42: 699,
         43: 584,
         44: 657,
         45: 623,
         46: 586,
         47: 583,
         48: 587,
         49: 497,
         50: 519,
         51: 483,
         52: 501,
         53: 459,
        

In [92]:
index_passages

defaultdict(set,
            {'radio as pour': {'JORFTEXT000000778230', 'JORFTEXT000023126211'},
             'assistance sociale et': {'JORFTEXT000000309114',
              'JORFTEXT000000504442',
              'JORFTEXT000000517504',
              'JORFTEXT000000571637',
              'JORFTEXT000000655739',
              'JORFTEXT000000657481',
              'JORFTEXT000000673309',
              'JORFTEXT000000723489'},
             'canalisatlon transport gaz': {'JORFTEXT000000163327'},
             'profit des notaires': {'JORFTEXT000000844759'},
             'jean roche ,': {'JORFTEXT000000621451'},
             'professeur de conservatoire': {'JORFTEXT000000832617'},
             'sous-prefete limoux -': {'JORFTEXT000028879392'},
             'relatif gestion oeuvres': {'JORFTEXT000000400379',
              'LEGITEXT000005629918'},
             'revenus professions independantes-declaration': {'JORFTEXT000000444719',
              'JORFTEXT000024144005'},
             'en norman

In [57]:
{a+1 for a in {1,2,3}}

{2, 3, 4}

In [6]:
connection = psycopg2.connect(dbname='loi', user='loi', password='baba')
curseur = connection.cursor()

curseur.execute("select bloc_textuel from article_2 where id_ = 'JORFARTI000001405662';")
resultats = curseur.fetchall()

curseur.close()
connection.close()


In [7]:
resultats

[("<CONTENU>  Par arrêté du ministre du travail et des affaires sociales en date du 12     juillet 1996, est fixé le modèle du formulaire de la &lt;&lt; déclaration des     revenus de l'année 1995 des professions indépendantes &gt;&gt; S 2223 g (1),<br />     enregistré par le Centre d'enregistrement et de révision des formulaires     administratifs (C.E.R.F.A.) sous le numéro 10020*01 ; la notice explicative     est enregistrée par le C.E.R.F.A. sous le numéro 5000201.<br />      L'arrêté du 5 février 1995, en tant qu'il fixait le précédent modèle S 2223     f (no C.E.R.F.A. 60-3944), est abrogé.<br />\n<br />\n</CONTENU>\n",)]